In [2]:
import pandas as pd
import nltk
from nltk.util import ngrams
from collections import Counter

In [2]:
reviews = pd.read_csv('sushi_reviews.csv')

In [3]:
tips = pd.read_csv('sushi_tips.csv')

In [4]:
tips

,Unnamed: 0,user_id,business_id,text,date,compliment_count,V6
0,1,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,2011-10-13 03:15:15,0,dont go dinner close 6 really yvonne l nega...
1,2,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...,2016-01-16 19:13:57,0,come early best service fill really quickly
2,3,xEM8BRJ4wm_s47Y0nL51MA,xv_U__GI9gmzKIHT_TANnw,"Best General Tso around. Also, try the grilled...",2011-07-30 22:54:31,0,best general tso around also try grilled swee...
3,4,cogPv5baWhsR-_Dcw3-Taw,-Xld662dL8WxrwP--bclIQ,The yum yum roll is delicious!,2011-05-01 01:48:56,0,yum yum roll delicious
4,5,jlN9luFsNZfZkxnM7cdLKQ,cv-SmPhbpwQCtlI2Q7KJIQ,Very good shanghai soup dumplings,2013-09-06 23:58:56,0,good shanghai soup dumplings
...,...,...,...,...,...,...,...
31567,31568,n28ACFmeqLzzTVpCP2m2Bw,xv_U__GI9gmzKIHT_TANnw,pepper steak combo. it was soso,2019-11-07 18:12:05,0,pepper steak combo soso
31568,31569,bHHTHTeM1Sb8-aun1pM8Tw,0OsR9lO16jxa0xWUY57s9g,Very good,2020-02-23 00:17:34,0,good
31569,31570,he-2lO3-bv9IhZw4ugKujQ,_OMGZ3TXOfN2By7skat_bw,Jarrod is Awesome. He tickles me! Dinner is great,2016-07-26 00:45:36,0,jarrod awesome tickles dinner great
31570,31571,UAIp182w-jpNXqLvu-M34A,DJFxBTBIgYErzKS5cA_LiQ,This is probably my favorite sushi bar ever! G...,2018-03-29 23:04:32,0,probably favorite sushi bar ever good food ...


In [18]:
%%time
bag = reviews['V10'].tolist()
bag = ' '.join(str(e) for e in bag)

Wall time: 17 ms


In [37]:
%%time
bigram = list(ngrams(nltk.word_tokenize(bag), 2))  #gen bigrams from bag
temp = dict(Counter(bigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Bi-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('bigrams_all_reviews.csv')

Wall time: 53.6 s


In [38]:
%%time
trigram = list(ngrams(nltk.word_tokenize(bag), 3))  #gen trigrams from bag
temp = dict(Counter(trigram))

df = pd.DataFrame(list(temp.items()))

df.columns = ['Tri-Gram', 'Frequency']
df = df.loc[df.Frequency > 5] #filter more frequent grams 5 mentions or more
df = df.sort_values(by=['Frequency'],ascending=False)
df.to_csv('trigrams_all_reviews.csv')

Wall time: 57.4 s


In [51]:
df_uni = pd.read_csv('unigrams_all_reviews.csv')
df_bi = pd.read_csv('bigrams_all_reviews.csv')
df_tri = pd.read_csv('trigrams_all_reviews.csv')


In [50]:
all_grams = pd.concat([df_uni, df_bi, df_tri], axis=1)
all_grams = all_grams.drop(['Unnamed: 0'], axis=1)

all_grams.to_csv('all_grams_all_reviews.csv', index = False)

# Filter Reviews by star and make n-gram tables 


In [79]:
%%time
#first loop for filter new dataset
for i in range(1,6): #stars
    current_star = f'{i}-Star'
    temp_df = reviews.loc[reviews.stars == i]
    
    bag = ''
    bag = temp_df['V10'].tolist()
    bag = ' '.join(str(e) for e in bag)

    #second loop for 1,2,3 grams
    for j in range(1,4):
        current_gram = f'{j}-Gram'
        grams = list(ngrams(nltk.word_tokenize(bag), j))  #gen j-Grams
        temp = dict(Counter(grams))
        if j == 1:
            gram_df = pd.DataFrame(list(temp.items()))
            gram_df.columns = [current_gram, 'Frequency']
            gram_df = gram_df.loc[gram_df.Frequency > 5] #filter more frequent grams 5 mentions or more
            gram_df = gram_df.sort_values(by=['Frequency'],ascending=False)
            
        else:
            temp = pd.DataFrame(list(temp.items()))
            temp.columns = [current_gram, 'Frequency']
            temp = temp.loc[temp.Frequency > 5] #filter more frequent grams 5 mentions or more
            temp = temp.sort_values(by=['Frequency'],ascending=False)
            temp.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
            gram_df.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
            gram_df = pd.concat([gram_df, temp], axis=1)
    gram_df.to_csv(f'{current_star}-grams.csv')

Wall time: 2min 47s


In [ ]:
%%time

stopwords = ['sushi', 'food', 'rolls', 'place', 'just', 'one']  #derived from overall n-gram table

for business_id in set(reviews['business_id']):
    #second loop to split reviews good v bad
    for k in ['good', 'bad']:
        if k == 'good': temp_df = reviews[(reviews.stars > 3) & (reviews.business_id == business_id)]
        else: temp_df = reviews[(reviews.stars < 3) & (reviews.business_id == business_id)]
        
        if len(temp_df['V10']) < 10: pass
        else:
        
            bag = ''
            bag = temp_df['V10'].tolist()
            bag = ' '.join(str(e) for e in bag)
            bag = nltk.word_tokenize(bag)
            tokens = [str(token) for token in bag if str(token) not in stopwords]
            #third loop for 1,2,3 grams
            for j in range(1,4):
                current_gram = f'{j}-Gram'
                
                grams = list(ngrams(tokens, j))  #gen j-Grams
                temp = dict(Counter(grams))
                if j == 1:
                    gram_df = pd.DataFrame(list(temp.items()))
                    gram_df.columns = [current_gram, 'Frequency']
                    gram_df[f'Percent Total {k} {j}-Grams'] = gram_df['Frequency']/sum(gram_df['Frequency'])
                    #gram_df = gram_df.loc[gram_df.Frequency > 5] #filter more frequent grams 5 mentions or more
                    gram_df = gram_df.sort_values(by=['Frequency'],ascending=False)
                else:
                    temp = pd.DataFrame(list(temp.items()))
                    temp.columns = [current_gram, 'Frequency']
                    temp[f'Percent Total {k} {j}-Grams'] = temp['Frequency']/sum(temp['Frequency'])
                    temp = temp.sort_values(by=['Frequency'],ascending=False)
                    temp.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
                    gram_df.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
                    gram_df = pd.concat([gram_df, temp], axis=1)
            
            gram_df.truncate(0, 99).to_csv(f'app_data/review-grams/{business_id}-{k}-grams.csv')    

# Filter Tips by business and make n-gram tables 




In [9]:
%%time

stopwords = ['sushi', 'food', 'rolls', 'place', 'just', 'one']  #derived from overall n-gram table

for business_id in set(tips['business_id']):
    temp_df = tips[tips.business_id == business_id]
        
    if len(temp_df['V6']) < 5: pass
    else:

        bag = ''
        bag = temp_df['V6'].tolist()
        bag = ' '.join(str(e) for e in bag)
        bag = nltk.word_tokenize(bag)
        tokens = [str(token) for token in bag if str(token) not in stopwords]
        #third loop for 1,2,3 grams
        for j in range(1,4):
            current_gram = f'{j}-Gram'

            grams = list(ngrams(tokens, j))  #gen j-Grams
            temp = dict(Counter(grams))
            if j == 1:
                gram_df = pd.DataFrame(list(temp.items()))
                gram_df.columns = [current_gram, 'Frequency']
                gram_df[f'Percent Total {j}-Grams'] = gram_df['Frequency']/sum(gram_df['Frequency'])
                gram_df = gram_df.sort_values(by=['Frequency'],ascending=False)
            else:
                temp = pd.DataFrame(list(temp.items()))
                temp.columns = [current_gram, 'Frequency']
                temp[f'Percent Total {j}-Grams'] = temp['Frequency']/sum(temp['Frequency'])
                temp = temp.sort_values(by=['Frequency'],ascending=False)
                temp.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
                gram_df.reset_index(drop=True, inplace=True)  #fix bug where indices cause ordering issue in df
                gram_df = pd.concat([gram_df, temp], axis=1)

        gram_df.truncate(0, 99).to_csv(f'app_data/tip-grams/{business_id}-grams.csv')    

Wall time: 28 s
